In [ ]:
# Copyright 2024 Google LLC
#
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch
import json
import random
from PIL import Image
import os
import argparse

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)


qasa_filtered_annotations_path = '../../../datasets/test-B/SPIQA_testB.json'
with open(qasa_filtered_annotations_path, "r") as f:
  qasa_data = json.load(f)


_QASA_IMAGE_ROOT = "../../../datasets/test-B/SPIQA_testB_Images"

def prepare_inputs(paper, question_idx):
    all_figures = list(paper['all_figures_tables'].keys())
    referred_figures = list(set(paper['referred_figures_tables'][question_idx]))
    answer = paper['composition'][question_idx]

    referred_figures_captions = []
    for figure in referred_figures:
        referred_figures_captions.append(paper['all_figures_tables'][figure])

    return answer, all_figures, referred_figures, referred_figures_captions


_PROMPT = ["""USER: <image>\n Caption: <caption> Is the input image and caption helpful to answer the following question. Answer in one word - Yes or No. Question: <question>.\nASSISTANT:""", 
           """USER: <image>\n Caption: <caption> Please provide a brief answer to the following question after looking into the input image and caption. Question: <question>.\nASSISTANT:"""]


def infer_llava(qasa_data, args):

  model_id = args.model_id
  processor = AutoProcessor.from_pretrained(model_id)
  model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

  _RESPONSE_ROOT = args.response_root
  os.makedirs(_RESPONSE_ROOT, exist_ok=True)

  for paper_id, paper in sorted(qasa_data.items(), key=lambda x: random.random()):
    if os.path.exists(os.path.join(_RESPONSE_ROOT, str(paper_id) + '_response.json')):
      continue
    response_paper = {}

    try:
      for question_idx, question in enumerate(paper['question']):

        answer, all_figures, referred_figures, referred_figures_captions = prepare_inputs(paper, question_idx)

        answer_dict = {}

        for _idx, figure in enumerate(referred_figures):
          
          caption = referred_figures_captions[_idx]

          contents = [_PROMPT[0].replace('<caption>', caption).replace('<question>', question),
                      _PROMPT[1].replace('<caption>', caption).replace('<question>', question)]
          
          # contents = _PROMPT.replace('<question>', question)
          image = Image.open(os.path.join(_QASA_IMAGE_ROOT, figure))
          image = image.resize((args.image_resolution, args.image_resolution))
          inputs = processor(contents, [image, image], padding=True, return_tensors="pt").to("cuda")
          # inputs = processor(contents, image, padding=True, return_tensors="pt").to("cuda")

          output = model.generate(**inputs, max_new_tokens=100)
          generated_text = processor.batch_decode(output, skip_special_tokens=True)

          answer_dict.update({figure: [generated_text[0].split("ASSISTANT:")[-1], generated_text[1].split("ASSISTANT:")[-1]]})
        
          print(answer_dict[figure])
          print('-----------------')

        question_key = paper['question_key'][question_idx]
        response_paper.update({question_key: {'question': question, 'response': answer_dict,
                                            'referred_figures_names': referred_figures, 'answer': answer}})   

    except:
      print('Error in generating.')
      processor = AutoProcessor.from_pretrained(model_id)
      model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")
      continue

    with open(os.path.join(_RESPONSE_ROOT, str(paper_id) + '_response.json'), 'w') as f:
      json.dump(response_paper, f)

if __name__ == '__main__':
    
    parser = argparse.ArgumentParser(description='Evaluate on Qasa/Qasper.')
    parser.add_argument('--model_id', type=str, default='llava-hf/llava-1.5-7b-hf', help='Huggingface Model id.')
    parser.add_argument('--response_root', type=str, help='Response Root path.')
    parser.add_argument('--image_resolution', type=int, help='Response Root path.')
    args = parser.parse_args()
    
    
    infer_llava(qasa_data, args)

In [2]:
! pip install transformers

  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.2 MB 1.0 MB/s eta 0:00:10
   --- ------------------------------------ 0.8/10.2 MB 1.1 MB/s eta 0:00:09
   ---- ----------------------------------- 1.0/10.2 MB 1.2 MB/s eta 0:00:08
   ----- ---------------------------------- 1.3/10.2 MB 1.3 MB/s eta 0:00:07
   ------- -------------------------------- 1.8/10.2 MB 1.5 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/10.2 MB 1.5 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/10.2 MB 1.5 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/10.2 MB 1.5 MB/s eta 0:00:06
   ---------- ---------------

In [3]:
# CPU friendly version
from transformers import AutoProcessor, LlavaForConditionalGeneration
# import torch


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import io
import base64
from tqdm import tqdm
import json
import random
from PIL import Image
import os
import argparse

In [24]:
from huggingface_hub import InferenceClient

In [ ]:

# Set this to your preferred hosted model on Hugging Face or your own endpoint
# API_URL = "https://api-inference.huggingface.co/models/llava-hf/llava-1.5-7b-hf"
# HF_TOKEN = "your_hf_token_here"  # if needed

# client = InferenceClient(model=API_URL, token=HF_TOKEN)


In [4]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1235/v1", api_key="lm-studio")

In [36]:
completion = client.chat.completions.create(
  model="llava-v1.5-7b",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "what can you do with images"}
  ],
  temperature=0.9,
)


In [35]:
try:
    print(completion.choices[0].message.content)
except:
    print("Error occured")
    

Error occured


In [5]:

qasa_filtered_annotations_path = '../../../../Data/spiqa/test-B/SPIQA_testB.json'
with open(qasa_filtered_annotations_path, "r") as f:
    qasa_data = json.load(f)

_QASA_IMAGE_ROOT = '../../../../Data/spiqa/test-B/Images/SPIQA_testB_Images/SPIQA_testB_Images'
# _QASA_IMAGE_ROOT = "#"../../../datasets/test-B/SPIQA_testB_Images"

_PROMPT = [
    """<image>\n Caption: <caption> Is the input image and caption helpful to answer the following question. Answer in one word - Yes or No. Question: <question>.\nASSISTANT:""",
    """<image>\n Caption: <caption> Please provide a brief answer to the following question after looking into the input image and caption. Question: <question>.\nASSISTANT:"""
]


In [6]:
def encode_image_to_base64(image_path):
    with Image.open(image_path) as img:
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode("utf-8")

In [7]:
def prepare_inputs(paper, question_idx):
    all_figures = list(paper['all_figures_tables'].keys())
    referred_figures = list(set(paper['referred_figures_tables'][question_idx]))
    answer = paper['composition'][question_idx]

    referred_figures_captions = []
    for figure in referred_figures:
        referred_figures_captions.append(paper['all_figures_tables'][figure])

    return answer, all_figures, referred_figures, referred_figures_captions

def infer_llava(qasa_data, args):
    _RESPONSE_ROOT = args.response_root
    # _QASA_IMAGE_ROOT = "../../../datasets/test-B/SPIQA_testB_Images"
    _QASA_IMAGE_ROOT = '../../../../Data/spiqa/test-B/Images/SPIQA_testB_Images/SPIQA_testB_Images'
    os.makedirs(_RESPONSE_ROOT, exist_ok=True)
    
    for paper_id, paper in tqdm(sorted(qasa_data.items(), key=lambda x: random.random()), desc="Processing papers", leave=False):
    # for paper_id, paper in sorted(qasa_data.items(), key=lambda x: random.random()):
        output_path = os.path.join(_RESPONSE_ROOT, f"{paper_id}_response.json")
        if os.path.exists(output_path):
            continue

        response_paper = {}

        try:
            for question_idx, question in enumerate(paper['question']):
                all_figures = list(paper['all_figures_tables'].keys())
                referred_figures = list(set(paper['referred_figures_tables'][question_idx]))
                answer = paper['composition'][question_idx]

                referred_figures_captions = [
                    paper['all_figures_tables'][fig] for fig in referred_figures
                ]

                answer_dict = {}

                for idx, figure in enumerate(referred_figures):
                    caption = referred_figures_captions[idx]
                    image_path = os.path.join(_QASA_IMAGE_ROOT, figure)
                    base64_image = encode_image_to_base64(image_path)

                    responses = []
                    for prompt_template in _PROMPT:
                        prompt = prompt_template.replace("<caption>", caption).replace("<question>", question)

                        result = client.chat.completions.create(
                            model="llava",  # Make sure this matches your LM Studio model name
                            messages=[
                                {
                                    "role": "user",
                                    "content": [
                                        {"type": "text", "text": prompt},
                                        {
                                            "type": "image_url",
                                            "image_url": {
                                                "url": f"data:image/png;base64,{base64_image}"
                                            }
                                        }
                                    ]
                                }
                            ],
                            temperature=0.2,
                            max_tokens=300
                        )

                        text = result.choices[0].message.content.strip()
                        responses.append(text)

                    answer_dict[figure] = responses
                    print(answer_dict[figure])
                    print('-----------------')

                question_key = paper['question_key'][question_idx]
                response_paper[question_key] = {
                    'question': question,
                    'response': answer_dict,
                    'referred_figures_names': referred_figures,
                    'answer': answer
                }

        except Exception as e:
            print(f"Error processing paper {paper_id}: {e}")
            continue

        with open(output_path, 'w') as f:
            json.dump(response_paper, f, indent=2)

In [8]:
image_path = os.path.join('C:\\Users\\Lisara\\RGU-Y4-C\\FYP\\FYIRP\\Notebooks\\datasets\\test-B\\SPIQA_testB_Images\\f37e90c0bd5c4a9619ccfb763c45cb2d84abd3e6\\3-Figure1-1.png')
    # _QASA_IMAGE_ROOT, figure)

if not os.path.exists(image_path):
    print(f"[Warning] Image not found: {image_path}")
    # continu


[Warning] Image not found: C:\Users\Lisara\RGU-Y4-C\FYP\FYIRP\Notebooks\datasets\test-B\SPIQA_testB_Images\f37e90c0bd5c4a9619ccfb763c45cb2d84abd3e6\3-Figure1-1.png


In [9]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Evaluate with LLaVA API on CPU.')
    parser.add_argument('--response_root', type=str, help='Response Root path.')
    parser.add_argument('--image_resolution', type=int, default=336, help='Image resolution.')

    # In Jupyter/interactive mode, use parse_known_args to avoid "--f=..." error
    args, _ = parser.parse_known_args()

    # Manually set values if they aren't passed via CLI (for Jupyter safety)
    args.response_root = "./responses"
    args.image_resolution = 336

    infer_llava(qasa_data, args)

Processing papers:   0%|          | 0/65 [00:00<?, ?it/s]

['No', 'Yes, α controls the strength of the length normalization, while β controls the strength of the coverage penalty. In this table, the values of α and β are varied to see how they affect the BLEU score of the model. The results show that varying these two parameters can have a significant impact on the performance of the model in terms of translation quality.']
-----------------
['No', "The attention mechanism connecting the bottom layer of the decoder to the top layer of the encoder in GNMT contributes to improving parallelism by allowing multiple GPUs to work on different parts of the model simultaneously. This is achieved through the use of attention modules, which enable the decoder to selectively focus on relevant information from the encoder's output. By doing so, the decoder can process multiple input sequences in parallel without requiring all layers to be computed sequentially. As a result, this enhances the overall efficiency and speed of training and inference tasks for

Processing papers:   2%|▏         | 1/65 [17:06<18:14:27, 1026.05s/it]

['Yes', "The reason for separating the constraint values of δ and γ is to ensure that the model's output remains within a certain range, which helps in preventing overfitting. The clipping constraints act as additional regularization that improves the model quality by limiting the model's ability to fit the training data too closely. This can lead to better generalization performance and improved overall model accuracy."]
-----------------
['No', 'The SBM-Transformer is considered better than Reformer because it uses a scaled dot product attention mechanism, which allows for more efficient and effective processing of the data. This results in improved performance and accuracy compared to traditional methods like Reformer. The image shows a diagram illustrating the process of using a scaled dot product attention mechanism, where the SBM is used to sample bipartite graphs connecting queries to keys from an underlying SBM. The adjacency of the sampled graph is then used as an attention ma

Processing papers:   3%|▎         | 2/65 [21:41<10:13:50, 584.61s/it] 

...

Processing papers:  97%|█████████▋| 63/65 [27:03:18<57:40, 1730.11s/it]  

['No', 'Was transfer learning beneficial in the CADe process? Yes, it was beneficial as shown by the table in the image. The table compares the accuracy of interstitial lung disease classification using both slice-level (SLICE-CV5) and patch-based (PATCH-CV5) classification methods with five-fold cross validation. The results show that the PATCH-CV5 method outperformed the SLICE-CV5 method in terms of accuracy, as indicated by the bold numbers in the table. This demonstrates that transfer learning was effective in improving the performance of the CADe process for interstitial lung disease classification.']
-----------------
['No', 'The table in the image displays various models of language processing, including LSTM-based models and non-neural models such as Kneser-Ney 5-gram. The table also shows the performance of these models on the English Penn Treebank test set, with perplexity (PPL) and size being used to measure their performance. It is evident that the LSTM-based models general

Processing papers:  98%|█████████▊| 64/65 [27:08:20<21:41, 1301.65s/it]

['No', 'The table shows the performance of various language models on the English Penn Treebank test set, with different numbers of parameters (size) and perplexity (PPL). The models include LSTM-WordSmall, LSTM-CharSmall, LSTM-Large, KN-5, and other neural models. The table shows the performance in terms of PPL and size for each model.\n\nThe LSTM-WordSmall has a perplexity of 97 and is smaller than all other models. The LSTM-CharSmall has a perplexity of 83 and is larger than the LSTM-WordSmall but smaller than the KN-5 model. The LSTM-Large has a perplexity of 61 and is larger than both the LSTM-CharSmall and the KN-5 model. The KN-5 has a perplexity of 47, which is the lowest among all models.\n\nThe table also shows that the LSTM-WordSmall is significantly better than the baseline (KN-5) by a factor of 9 times in terms of perplexity.']
-----------------
['Yes', 'Question: What does "information highways" mean?']
-----------------
['No', 'The authors claim that their LSTM network s

['No', 'The image shows a comparison between two types of neural networks - highway and plain networks. The left side of the image displays the training curves for various depths of both network types, while the right side presents the mean performance of top 10 hyperparameter settings for each network type. It can be observed that the highway networks with up to 100 layers still perform well despite their increasing depth. On the other hand, plain networks become harder to optimize as they increase in depth. This highlights the benefits of using highway networks over plain networks, especially when dealing with large datasets and complex problems.']
-----------------


In [ ]:
# 114 min for 25%1356 80
# 186 for 34%
# 359 51
# 430 54%
# 548 63
# 784 74
# 1552 92%
#  1642
